In [11]:
import polars as pl
from pathlib import Path


# load pareto sets
# problem_name = "branin"
problem_name = "mystery"
file_ext = "parquet"
n_gen_front = 1_000
psize_front = 100
# optimal_value = -268.78792  # branin
optimal_value = -1.174261  # mystery

fronts_path = Path("results/fronts/")
front_file_name = Path(fronts_path / f"{problem_name}_gen{n_gen_front}_psize{psize_front}.{file_ext}")

df_front = pl.read_parquet(front_file_name)
df_front.filter(pl.col("c_1") <= 0).sort("f_1")

x_1,x_2,f_1,c_1,f_1_min,c_1_min,generation
f64,f64,f64,f64,f64,f64,i32
2.745334,2.351738,-1.173179,-0.000897,-1.173179,-0.000897,693
2.747312,2.353608,-1.172896,-0.001005,-1.172896,-0.001005,602
2.745308,2.35136,-1.172745,-0.001249,-1.172745,-0.001249,675
2.743394,2.349178,-1.172166,-0.001517,-1.172166,-0.001517,528
2.746141,2.351371,-1.171743,-0.002072,-1.171743,-0.002072,130
…,…,…,…,…,…,…
3.695044,1.733676,4.266763,-0.999998,4.266763,-0.999998,688
3.685692,1.723512,4.266767,-0.999999,4.266767,-0.999999,190
3.689845,1.726443,4.274481,-1.0,4.274481,-1.0,974


In [12]:
f_col = "f_1_min"
c_col = "c_1_min"

front = df_front.select([f_col, c_col]).drop_nulls().filter(pl.col(f_col).is_finite() & pl.col(c_col).is_finite())

f_ideal, f_nadir = front.select(
    pl.min(f_col).alias("f_ideal"),
    pl.max(f_col).alias("f_nadir"),
)

front_norm = front.with_columns(
    ((pl.col(f_col) - f_ideal) / (f_nadir - f_ideal)).alias("f_norm"),
)

norm_optimal_value = (optimal_value - f_ideal) / (f_nadir - f_ideal)
norm_optimal_value

f_ideal
f64
0.017236


In [13]:
low_th_norm = norm_optimal_value - 0.01 * norm_optimal_value
med_th_norm = norm_optimal_value - 0.05 * norm_optimal_value
hig_th_norm = norm_optimal_value - 0.10 * norm_optimal_value

print(med_th_norm)

deltas = front_norm.with_columns(
    (abs(pl.col("f_norm") - low_th_norm)).alias("low_delta"),
    (abs(pl.col("f_norm") - med_th_norm)).alias("med_delta"),
    (abs(pl.col("f_norm") - hig_th_norm)).alias("hig_delta"),
)

low_ct = deltas.sort("low_delta").select(pl.first(c_col))
med_ct = deltas.sort("med_delta").select(pl.first(c_col))
hig_ct = deltas.sort("hig_delta").select(pl.first(c_col))

print(f"{low_ct=}\n, {med_ct=}\n, {hig_ct=}\n")


shape: (1,)
Series: 'f_ideal' [f64]
[
	0.016374
]
low_ct=shape: (1, 1)
┌──────────┐
│ c_1_min  │
│ ---      │
│ f64      │
╞══════════╡
│ 0.002758 │
└──────────┘
, med_ct=shape: (1, 1)
┌─────────┐
│ c_1_min │
│ ---     │
│ f64     │
╞═════════╡
│ 0.01188 │
└─────────┘
, hig_ct=shape: (1, 1)
┌──────────┐
│ c_1_min  │
│ ---      │
│ f64      │
╞══════════╡
│ 0.023826 │
└──────────┘

